In [ ]:
import cv2
import numpy as np

def process_frame(frame):
    # Convert to HSV color space to isolate yellow and white colors
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # Define color thresholds for yellow
    yellow_lower = np.array([15, 100, 100], dtype=np.uint8)
    yellow_upper = np.array([35, 255, 255], dtype=np.uint8)
    
    # Create a mask for yellow
    mask_yellow = cv2.inRange(hsv, yellow_lower, yellow_upper)
    
    # Convert the frame to grayscale for white line detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Apply a binary threshold to detect white lines
    _, mask_white = cv2.threshold(gray, 155, 255, cv2.THRESH_BINARY)
    
    # Combine the yellow and white masks
    combined_mask = cv2.bitwise_or(mask_yellow, mask_white)
    
    # Apply Canny edge detection
    threshold_low = 10
    threshold_high = 210
    edges = cv2.Canny(combined_mask, threshold_low, threshold_high)
    
    # Define vertices for region of interest (ROI)
    height, width = frame.shape[:2]
    vertices = np.array([[(249, 410), (367, 359), (407, 357), (504, 409)]], dtype=np.int32)
    
    mask = np.zeros_like(edges)   
    cv2.fillPoly(mask, vertices, 255)
    masked_edges = cv2.bitwise_and(edges, mask)
    
    # Hough Line Transform to detect both straight lines and line segments
    rho = 2            # distance resolution in pixels 
    theta = np.pi/180  # angular resolution in radians 
    threshold = 20     # minimum number of votes 
    min_line_len = 20  # minimum number of pixels making up a line
    max_line_gap = 300  # maximum gap in pixels between connectable line segments    
    lines = cv2.HoughLinesP(masked_edges, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    
    # Create an empty black image to draw lines on
    line_image = np.zeros((frame.shape[0], frame.shape[1], 3), dtype=np.uint8)
    
    if lines is not None:
        for line in lines:
            for x1, y1, x2, y2 in line:      
                cv2.line(line_image, (x1, y1), (x2, y2), [0, 255, 0], 10)  # Green color [0, 255, 0] and thicker line (10)
    
    # Combine the original frame with the line image
    α = 1  # Weight of the original image
    β = 1  # Weight of the lines image
    γ = 0  # Scalar added to each sum
    Image_with_lines = cv2.addWeighted(frame, α, line_image, β, γ)
    
    return Image_with_lines

def main():
    input_video_path = r'C:\Users\Omen\Downloads\Lane detection lectures\test_videos\02d5e1be-cb00355e.mov'  # Path to the input video
    output_video_path = r'C:\Users\Omen\Downloads\Lane detection lectures\test_videos_output\02d5e1be-cb00355e.mov12345.mov'  # Path to save the output video
    
    cap = cv2.VideoCapture(input_video_path)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        processed_frame = process_frame(frame)
        out.write(processed_frame)
    
    cap.release()
    out.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


In [ ]:
import cv2
import numpy as np

# Initialize the list of points (ROI vertices)
roi_vertices = [(20, 460), (340, 300), (460, 300), (740, 460)]
selected_vertex = -1  # Index of the selected vertex
radius = 5  # Radius for showing the vertices
dragging = False  # Flag to indicate if dragging is occurring

def draw_polygon(image, vertices):
    """ Draw the polygon with the current vertices on the image """
    # Draw the ROI
    pts = np.array(vertices, np.int32)
    pts = pts.reshape((-1, 1, 2))
    cv2.polylines(image, [pts], isClosed=True, color=(0, 255, 0), thickness=2)
    
    # Draw the vertices
    for (x, y) in vertices:
        cv2.circle(image, (x, y), radius, (0, 0, 255), -1)

def mouse_callback(event, x, y, flags, param):
    global roi_vertices, selected_vertex, dragging
    
    if event == cv2.EVENT_LBUTTONDOWN:
        # Check if the mouse is clicked near any of the vertices
        for i, (vx, vy) in enumerate(roi_vertices):
            if abs(x - vx) < radius and abs(y - vy) < radius:
                selected_vertex = i
                dragging = True
                break
    
    elif event == cv2.EVENT_MOUSEMOVE and dragging:
        # Drag the selected vertex
        if selected_vertex != -1:
            roi_vertices[selected_vertex] = (x, y)
    
    elif event == cv2.EVENT_LBUTTONUP:
        # Release the selected vertex
        dragging = False
        selected_vertex = -1

def interactive_roi_selection(image):
    global roi_vertices
    
    # Create a copy of the image for displaying
    display_image = image.copy()
    
    # Create a window and set the mouse callback function
    cv2.namedWindow("Interactive ROI Selection")
    cv2.setMouseCallback("Interactive ROI Selection", mouse_callback)
    
    while True:
        # Display the image with the current ROI
        temp_image = display_image.copy()
        draw_polygon(temp_image, roi_vertices)
        cv2.imshow("Interactive ROI Selection", temp_image)
        
        # Break the loop when the user presses the 'q' key
        key = cv2.waitKey(1)
        if key == ord('q'):
            break
    
    cv2.destroyAllWindows()
    return roi_vertices

def main():
    input_video_path = r'C:\Users\Omen\Downloads\Lane detection lectures\test_videos\02d5e1be-cb00355e.mov'  # Path to the input video
    
    # Open the video file
    cap = cv2.VideoCapture(input_video_path)
    
    if not cap.isOpened():
        print(f"Error: Unable to open video at {input_video_path}. Please check the file path.")
        return
    
    # Read the first frame (or any specified frame)
    ret, frame = cap.read()
    
    if not ret:
        print("Error: Unable to read a frame from the video.")
        return
    
    # Run the interactive ROI selection tool on the extracted frame
    updated_vertices = interactive_roi_selection(frame)
    
    print("Updated ROI vertices:")
    print(updated_vertices)
    
    # Now, you can use `updated_vertices` in your processing code
    # Convert the vertices to a NumPy array for further processing
    vertices = np.array([updated_vertices], dtype=np.int32)

    # Example of using the vertices in a mask
    mask = np.zeros_like(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY))
    cv2.fillPoly(mask, vertices, 255)
    
    # Show the masked region
    masked_image = cv2.bitwise_and(frame, frame, mask=mask)
    cv2.imshow("Masked Image", masked_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    # Release the video capture object
    cap.release()

if __name__ == "__main__":
    main()
